In [103]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import seaborn as sns
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [115]:
print('Baseline conversion:') 
baseline_conversion = input()

print('Power')
power = input()

print('Significance alpha')
alpha = input()

print('MDE:')
mde = input()

print('Ratio Test/Control')
ratio = input()


Baseline conversion:


 0.2


Power


 0.8


Significance alpha


 0.05


MDE:


 0.02


Ratio Test/Control


 1


In [117]:
effect_size = sms.proportion_effectsize(float(baseline_conversion), float(baseline_conversion) + float(mde))
sample_size = sms.NormalIndPower().solve_power(effect_size, power = float(power), alpha = float(alpha), ratio = float(ratio)) 
sample_size = ceil(sample_size)
sample_size

6508

In [119]:
#Here should be import result of test for github simulations have:
print('Simulations test-conversions rate (baseline: {}, MDE: {}):'.format(baseline_conversion, mde))
probably_test_conversion = input()

control_group_conversions = list(np.random.binomial(1, float(baseline_conversion), int(sample_size)))
test_group_conversions = list(np.random.binomial(1, float(probably_test_conversion), int(sample_size)))

df_control = pd.DataFrame({'sample': int(sample_size)*['control'], 
                           'results': control_group_conversions})
df_test = pd.DataFrame({'sample': int(sample_size)*['test'], 
                           'results': test_group_conversions})

df_estimate = pd.concat([df_control, df_test])
df_estimate = df_estimate.sample(frac=1).reset_index(drop=True)
df_estimate


Simulations test-conversions rate (baseline: 0.2, MDE: 0.02):


 0.21


,sample,results
0,control,1
1,test,0
2,test,0
3,test,0
4,test,0
...,...,...
13011,control,0
13012,test,0
13013,control,1
13014,control,1


In [121]:
#Estimate Standart deviation and Standart Error: 

def std_dev(x):
    return np.std(x, ddof=0)

def std_error(x):
    return stats.sem(x, ddof=0)


conversions_df = df_estimate.groupby(['sample']).agg([np.mean, std_dev, std_error])
conversions_df

results                    
             mean   std_dev std_error
sample                               
control  0.204364  0.403236  0.004998
test     0.217425  0.412494  0.005113

In [123]:
num_control = int(sample_size)
num_test = int(sample_size)

conversions = [df_control['results'].sum(), df_test['results'].sum()]
nobs = [num_control, num_test]
z_stat, p_value = proportions_ztest(conversions, nobs = nobs)
(lower_control, lower_test), (upper_control, upper_test) = proportion_confint(conversions, nobs=nobs)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {p_value:.10f}')
print(f'ci 95% for control group: [{lower_control:.3f}, {upper_control:.3f}]')
print(f'ci 95% for test group: [{lower_test:.3f}, {upper_test:.3f}]')


z statistic: -1.83
p-value: 0.0677999969
ci 95% for control group: [0.195, 0.214]
ci 95% for test group: [0.207, 0.227]
